In [28]:
from dnadb import dna, fasta
from deepdna.nn.models import load_model
from deepdna.nn.models import setbert
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import wandb

In [44]:
with open("./logs/sequences.pkl", "rb") as f:
    sample_sequences = pickle.load(f)

In [30]:
sequences_db = fasta.FastaDb("./SnakeExperiment2/sequences.fasta.db/")

In [31]:
api = wandb.Api()

In [32]:
path = "sirdavidludwig/taxonomy-classification/setbert-taxonomy-topdown-silva-nr99-filtered-515f-806r-64d-150bp:v2"
model = load_model(api.artifact(path).download(True))

wandb: Downloading large artifact setbert-taxonomy-topdown-silva-nr99-filtered-515f-806r-64d-150bp:v2, 65.78MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.3


In [34]:
sequences_db[sample_sequences["c2"][0][0][0]]

FastaEntry(sequence='GTGCCAGCAGCCGCGGTAATACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGGGTGCGTAGGTGGCTCTTTAAGTGGGATGTGAAAGCCCTTGGCTCAACCAAGGAACTGCATTCCAAACTGGAGAGCTAGAGTGCAGGAGAGGAAAGTGGAATTCCTAGTGTAGCGGTGAAATGCGTAGAGATTAGGAGGAACACCAGTGGCGAAGGCGACTCGCTGGACTGATACTGACGCTGAGGTGCG', identifier='10409253', extra='')

In [45]:
confidence = 0.0
sample_taxonomies = {}
for sample_name, samples in tqdm(sample_sequences.items()):
    sample_taxonomies[sample_name] = []
    for sample in samples:
        sample = sample[0]
        entries = [sequences_db[id] for id in sample]
        entries = [e for e in entries if len(e.sequence) >= 150]
        sequences = np.array([dna.encode_sequence(e.sequence[:150]) for e in entries])
        assert np.all(sequences < 5)
        sequences = dna.encode_kmers(sequences, 3)
        result = model.predict(sequences[None,:,:], verbose=False)[0]
        result = dict(zip([e.identifier for e in entries], [r.constrained_taxonomy(confidence) for r in result]))
        sample_taxonomies[sample_name].append(result)

100%|██████████| 14/14 [00:55<00:00,  3.98s/it]


In [46]:
with open("./logs/taxonomies.pkl", "wb") as f:
    pickle.dump(sample_taxonomies, f)